In [1]:
from openai import AsyncOpenAI
from dotenv import load_dotenv
import os

In [2]:
def hybrid_refine_script(script):
    # 제거하거나 요약할 상담사의 키워드
    noise_keywords = ["성함", "생년월일", "번호", "핸드폰", "본인", "님"]
    wait_keywords = ["잠시만", "기다려", "조회", "확인", "맞습니까?"]
    
    lines = script.split('\n')
    refined_lines = []
    skip_counselor_tag_added = False # 태그 중복 방지용
    
    for line in lines:
        line = line.strip()
        if not line: continue
        
        # [손님 발화] -> 무조건 보존
        if "손님:" in line:
            refined_lines.append(line)
            skip_counselor_tag_added = False
            
        # [상담사 발화] -> 필터링 및 요약
        elif "상담사:" in line:
            content = line.replace("상담사:", "").strip()
            
            # 본인 확인이나 단순 대기 요청인 경우 -> 요약 태그로 치환
            if any(k in content for k in noise_keywords + wait_keywords):
                if not skip_counselor_tag_added:
                    refined_lines.append("[상담사: 본인 확인 또는 정보 안내]")
                    skip_counselor_tag_added = True # 연속된 노이즈는 한 줄로 처리
            
            # 상담사의 설명이 20자 이상인 경우 (중요한 안내일 확률 높음)
            elif len(content) > 20:
                refined_lines.append(line)
                skip_counselor_tag_added = False
                
            # 너무 짧은 리액션(네, 네네 등)은 아예 삭제
            else:
                continue
                
    return "\n".join(refined_lines)

In [3]:
raw_script = """
상담사: 카드 상담원 ▲▲▲입니다.
손님: 안녕하세요?
상담사: 네, 안녕하십니까? 무엇을 도와드릴까요?
손님: 아 ▲▲일 날 무인 택시 선결제가 됐었는데 어제 자동 이체 출금이 이중 인출되고 또 계속해서 문자가 날아오네요.
상담사: 네, 아 그러십니까? 제가 한번 확인해 보겠습니다.
손님: 입금 안 됐다는 식으로 아까 낮에 통화했어요, 다른 분이랑.
상담사: 아 네.
손님: 왜 자꾸 이런 문자가 와요? 그게 결정이 안 된 것처럼.
상담사: 네, 확인 한번 해 보겠습니다. ▲▲▲ 님 본인 맞습니까?
손님: 예.
상담사: 네, 확인 감사드립니다. 네, 지금 ▲▲▲▲▲원이 이중 출금된 금액으로 확인되는데요, 이거 오늘 오전 중으로 결제 계좌 ▲▲은행으로 자동으로 환불 중에 있는 걸로 확인됩니다.
손님: 아까 그걸 그 다른 번호로 통화를 했는데 왜 자꾸 연체된 게 문자로 계속 오냐고요, 지금 잔액 부족이라는 식으로요.
상담사: 네, 잔액 부족이라고 오늘 지금 ▲▲카드에서 문자를 받으셨다는 거세요?
손님: 예, 하나 날아왔고 평일 기준 ▲▲일 출금됐다는 식으로 제가 또 받았어요.
상담사: 아 그러셨어요?
손님: 아니 선결제를 미리 했는데.
상담사: 아 네, 어제 날짜로 계좌에서 출금된 내역이 있었다고 이거는 자동으로 알림 발송이 된 걸로 확인되세요.
손님: 아니, 그 지금 뭐 자기네들이 다시 해야 될 게 빼간 돈을 넣지도 않고 있어요, 생각할수록 자꾸 짜증나게.
상담사: 아 이거는 어제 날짜 기준으로 자동 문자 발송된 거라서요, 이 금액이 출금 돼 있었다라고 그냥 알려드리는 거예요.
손님: 연체 이런 것도 안 뜨죠?
상담사: 네, 연체는 아닙니다.
손님: 그 핸드폰이랑 저기 ▲▲카드로 잘 안 되는데요, 지금 이게.
상담사: ▲▲은행인가 이게 결제 같은 경우는 ▲▲에 한 번씩 출금되기 때문에 결제일은 영업일 이전에 미리 선결제하더라도 결제일 당일에는 계좌 잔고가 있으면 이중 출금이 되실 수 있으세요.
손님: 아 그러니까 알았어요.
상담사: 예, 예.
상담사: 상담원 ▲▲▲이었습니다.

"""
print(hybrid_refine_script(raw_script))

손님: 안녕하세요?
상담사: 네, 안녕하십니까? 무엇을 도와드릴까요?
손님: 아 ▲▲일 날 무인 택시 선결제가 됐었는데 어제 자동 이체 출금이 이중 인출되고 또 계속해서 문자가 날아오네요.
[상담사: 본인 확인 또는 정보 안내]
손님: 입금 안 됐다는 식으로 아까 낮에 통화했어요, 다른 분이랑.
손님: 왜 자꾸 이런 문자가 와요? 그게 결정이 안 된 것처럼.
[상담사: 본인 확인 또는 정보 안내]
손님: 예.
[상담사: 본인 확인 또는 정보 안내]
손님: 아까 그걸 그 다른 번호로 통화를 했는데 왜 자꾸 연체된 게 문자로 계속 오냐고요, 지금 잔액 부족이라는 식으로요.
상담사: 네, 잔액 부족이라고 오늘 지금 ▲▲카드에서 문자를 받으셨다는 거세요?
손님: 예, 하나 날아왔고 평일 기준 ▲▲일 출금됐다는 식으로 제가 또 받았어요.
손님: 아니 선결제를 미리 했는데.
[상담사: 본인 확인 또는 정보 안내]
손님: 아니, 그 지금 뭐 자기네들이 다시 해야 될 게 빼간 돈을 넣지도 않고 있어요, 생각할수록 자꾸 짜증나게.
상담사: 아 이거는 어제 날짜 기준으로 자동 문자 발송된 거라서요, 이 금액이 출금 돼 있었다라고 그냥 알려드리는 거예요.
손님: 연체 이런 것도 안 뜨죠?
손님: 그 핸드폰이랑 저기 ▲▲카드로 잘 안 되는데요, 지금 이게.
상담사: ▲▲은행인가 이게 결제 같은 경우는 ▲▲에 한 번씩 출금되기 때문에 결제일은 영업일 이전에 미리 선결제하더라도 결제일 당일에는 계좌 잔고가 있으면 이중 출금이 되실 수 있으세요.
손님: 아 그러니까 알았어요.


---

In [ ]:
SYSTEM_PROMPT = """
상담 스크립트에서 고객의 성향을 분류하세요
전체적인 맥락을 참고하되 판단의 근거는 고객의 발화에 한정합니다

### 분류 규칙
1. 제시된 성향 키워드 중 가장 적절한 한가지만 선택
2. 설명이나 판단 근거는 절대 출력하지말고 오직 하나의 키워드만 출력한다
3. 여러 개의 성향을 가질 경우 S3 > S2 > S1 > N3 > N2 > N1 의 순서로 우선 순위를 가진다

### 성향 키워드 목록
- N1 (실용주의형): 불필요한 말 없이 바로 문의사항을 말함
- N2 (수다형): 사적인 이야기나 본인 상황을 길게 설명함
- N3 (신중형): 신중하고 의심을 보임
- S1 (급한성격형): 빠른 처리를 선호함
- S2 (이해부족형): 설명을 잘 이해하지 못하여 반복적으로 확인함
- S3 (불만형): 분노, 짜증을 드러냄
"""

In [5]:
import time
import openai

def get_personality_gpt(script):
    start_time = time.perf_counter()
    ttft = 0
    generated_tokens = 0
    content = ""
    first_token_received = False
    
    try:
        response = openai.chat.completions.create(
           model="gpt-4.1-mini",
           messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": script}
            ],
            temperature=0,
            stream=True,                              # 토큰 단위로 수신
            stream_options={"include_usage": True}    # 마지막 chunk에 토큰 정보 포함
        )
        
        for chunk in response:
            # 첫번째 토큰이 들어오는 시점 확인 (ttft)
            if not first_token_received and chunk.choices and chunk.choices[0].delta.content:
                ttft = time.perf_counter() - start_time
                first_token_received = True
            
            # 내용 누적
            if chunk.choices and chunk.choices[0].delta.content:
                content += chunk.choices[0].delta.content
            
            # 토큰 사용량 확인
            if chunk.usage is not None:
                generated_tokens = chunk.usage.completion_tokens
        
        end_time = time.perf_counter()
        total_duration = end_time - start_time
        
        # tps 계산 (생성된 토큰 수 / 전체 소요 시간)
        tps = generated_tokens / total_duration if total_duration > 0 else 0
        
        metrics = {
                "ttft": round(ttft, 3),
                "tps": round(tps, 2),
                "total_tokens": generated_tokens
            }

        return content, metrics
    except Exception as e:
        return f"호출 중 오류 발생 : {e}", "오류"

In [6]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import time

load_dotenv()

client = OpenAI(
    base_url=os.getenv("RUNPOD_URL"),
    api_key=os.getenv("API_KEY")
)

def get_personality_sllm(script):
    try:
        start_time = time.perf_counter() # 전체 시작 시간
        first_token_time = None
        full_content = ""
        
        # 스트리밍 호출 활성화
        response = client.chat.completions.create(
            model="local-model",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": script}
            ],
            temperature=0.0,
            stream=True,                            # 스트리밍 활성화
            stream_options={"include_usage": True}  # 토큰 사용량 포함
        )

        total_tokens = 0
        
        for chunk in response:
            # TTFT 계산 (첫 번째 토큰이 들어온 시점)
            if first_token_time is None and chunk.choices and chunk.choices[0].delta.content:
                first_token_time = time.perf_counter()
            
            # 내용 합치기
            if chunk.choices and chunk.choices[0].delta.content:
                full_content += chunk.choices[0].delta.content
            
            # 토큰 수
            if chunk.usage:
                total_tokens = chunk.usage.completion_tokens

        end_time = time.perf_counter()
        
        # 지표 계산
        ttft = first_token_time - start_time if first_token_time else 0
        total_latency = end_time - start_time
        
        # TPS 계산 (생성된 토큰 수 / 생성에 걸린 시간)
        if total_tokens == 0:
            total_tokens = len(full_content) / 1.5 
            
        tps = total_tokens / total_latency if total_latency > 0 else 0
        
        metrics = {
            "ttft": round(ttft, 3),
            "tps": round(tps, 2),
            "total_tokens": int(total_tokens)
        }

        return full_content, metrics

    except Exception as e:
        return {"error": f"오류 발생: {str(e)}"}, {"ttft": 0, "tps": 0, "total_tokens": 0}

In [7]:
import pandas as pd 

# 파일 경로 설정
testset = "hana_personality.csv"
results_list = []

df = pd.read_csv(testset)
data_list = df.to_dict('records')
    
for item in data_list:
    # 데이터 추출
    script = item.get('consulting_content', "")
    truth = item.get('personality', "")

    print(f"테스트 실행 중: {item.get('source_id', 'Unknown ID')}")
    # res, metrics = get_personality_gpt(script)
    res, metrics = get_personality_sllm(script)
        
    # 결과 저장
    results_list.append({
        "id": item.get('source_id'),
        "truth": truth,
        "res": res,
        "metrics": metrics,
    })

테스트 실행 중: 21461
테스트 실행 중: 21708
테스트 실행 중: 22949
테스트 실행 중: 23072
테스트 실행 중: 24399
테스트 실행 중: 24702
테스트 실행 중: 24754
테스트 실행 중: 26858
테스트 실행 중: 520284
테스트 실행 중: 437292
테스트 실행 중: 928837
테스트 실행 중: 107184
테스트 실행 중: 20691
테스트 실행 중: 21899
테스트 실행 중: 21045
테스트 실행 중: 21142
테스트 실행 중: 21197
테스트 실행 중: 21450
테스트 실행 중: 21499
테스트 실행 중: 21586
테스트 실행 중: 200532
테스트 실행 중: 27105
테스트 실행 중: 26084
테스트 실행 중: 26073
테스트 실행 중: 25782
테스트 실행 중: 25594
테스트 실행 중: 25506
테스트 실행 중: 20617
테스트 실행 중: 20748
테스트 실행 중: 20795
테스트 실행 중: 20992
테스트 실행 중: 21105
테스트 실행 중: 21381
테스트 실행 중: 21841
테스트 실행 중: 22201
테스트 실행 중: 22540
테스트 실행 중: 22556
테스트 실행 중: 22834
테스트 실행 중: 23649
테스트 실행 중: 23880
테스트 실행 중: 23977
테스트 실행 중: 24169
테스트 실행 중: 24252
테스트 실행 중: 25937
테스트 실행 중: 20868
테스트 실행 중: 20869
테스트 실행 중: 21202
테스트 실행 중: 21221
테스트 실행 중: 21272
테스트 실행 중: 21662
테스트 실행 중: 21834
테스트 실행 중: 22090
테스트 실행 중: 22272
테스트 실행 중: 22552
테스트 실행 중: 22589
테스트 실행 중: 22871
테스트 실행 중: 23160
테스트 실행 중: 23749
테스트 실행 중: 200292
테스트 실행 중: 26922
테스트 실행 중: 21126
테스트 실행 중: 21407
테스

In [8]:
import os
import pandas as pd

df_new = pd.json_normalize(results_list)
df_new['model'] = "Qwen2.5-7B" 

output_file = "evaluation_results_personality.csv"

if not os.path.exists(output_file):
    df_new.to_csv(output_file, index=False, mode='w', encoding='utf-8-sig')
else:
    df_new.to_csv(output_file, index=False, mode='a', encoding='utf-8-sig', header=False)

print(f"데이터가 추가되었습니다: {output_file}")

데이터가 추가되었습니다: evaluation_results_personality.csv
